# CH04 신경망 학습

## 4.1 데이터에서 학습한다

**학습** : 훈련데이터로부터 가중치 매개변수의 최적값을 자동으로 획득하는 것  
**손실함수** : 신경망이 학습할 수 있도록하는 지표  
기계학습은 인간의 개입을 최대한 배제라고 숨은 규칙성을 파악하는 데에 중점을 둠 
  
접근법  
* **(고전)머신러닝** : 사람이 생각한 특징을 svm등의 모델에 먹여 학습
* **신경망(딥러닝)** : 기계가 알아서 데이터를 관찰하고 숨겨진 특성 찾음
  
머신러닝은 범용성을 확보하기 위해 데이터를 훈련데이터와 시럼데이터로 나눔  
**범용성이란? 수집하진 못한 데이터에도 적용이 돼야함**

## 4.2 손실함수

**손실함수**: 신경망 성능의 나쁨을 나타내는 지표로 이 지표를 낮추는 것이 머신러닝의 과제 (음수처리하여 얼마나 좋은지에 대한 지표로도 사용가능)  

**오차제곱합**: Sum Squard Error  
말그대로 오차를 제곱해서 시그마 때린 것  
$$ E = \frac{1}{2}\sum_{k}(y_k-t_k)^2 $$
$$ y_k = 신경망의 추정 값$$
$$ t_k = 정답레이블 $$
$$k = 데이터의 차원 수(쉽게 말해 출력층의 노드 수)$$

구현 시

In [1]:
def sum_squared_error(y,t):
    return 0.5

**교차엔트로피 오차**  
CrossEntropyError (CEE)  
주로 분류에 사용  
$$ E = -\sum_{k}t_klog(y_k) $$
$$ t_k = \text{정답레이블} $$
$$ y_k = \text{모델예측값} $$
$$ k = \text{출력층의 노드 수} $$

**미니배치학습을 활용하여 미니배치에CEE 적용하기**  
미니배치 : 훈련 데이터에 대한 손실함수를 모든데이터에 대해 구하기 힘들때 **데이터의 일부를 추려 하나의 데이터처럼 이용** (Batch의 개념)  
즉, 데이터 일부를 추려 전체의 근사치로 활용하는 것  
$$E = - \frac{1}{N} \sum_{n} \sum_{k} t_{nk} \log y_{nk}$$

| 구분 | Step 1 (입력) | Step 2 (오차 계산) | Step 3 (학습) | Step 4 (반복) |
| --- | --- | --- | --- | --- |
| **CEE** | 데이터 하나 넣음<br>`(1 x 784)` | 망에 넣어 오차 얻음 | 가중치 수정 | 다음 데이터 넣고 반복 |
| **미니배치 CEE** | 데이터 배치 넣음<br>`(N x 784)` | 망에 넣어 CEE 오차들을 얻고<br>평균내서 새 오차 얻음 | 가중치 수정 | 다음 배치 넣고 반복 |

In [2]:
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist
from PIL import Image

(x_train, t_train), (x_test, t_test) = \
    load_mnist(flatten = True, normalize = True, one_hot_label=True)
x_train.shape

(60000, 784)

In [3]:
def cross_entropy_error(y,t):
    if y.ndim == 1:
        t = t.reshape(1,t.size)
        y = y.reshape(1,y.size)
    batch_size = y.shape[0]
    return -np.sum(mp.lof(y[np.arange(batch_size),t] + 1e-7)) / batch_size

## 4.3 수치미분

$ \frac{f(x + h) - f(x)}{h} $ 로 미분 구현 시 두가지 문제 내포
* 반올림 오차 : 작은 값 무시하므로 h->0 하기 힘듦
* 차분 : 점$x+h$ 와 점$x$ 사이의 기울기를 구한 것이지 미분계수를 구한 것이 아니기에 오차가 생김(h->0 구현의 한계)

**중심 차분**으로 오차를 줄임  
$ \frac{f(x + h) - f(x - h)}{2h} $  

In [4]:
def numerical_difference(f,x):
    h = 1e-4
    return (f(x+h) - f(x-h)) / (2*h)

In [5]:
#수치 미분의 예
def function_1(x):
    return 0.01*x**2 + 0.1*x

numerical_difference(function_1, 5)

0.1999999999990898

편미분  
$ f(x_0,x_1) = x_{0}^2 + x_{1}^2 $의 식일 때 변수가 2개이므로 주의

In [6]:
def function_2(x):
    return x[0]**2+x[1]**2

#### 함수 사용 예문

문제:  
$ x_0 = 3, x_1 = 4 $ 일때, $ x_0 $에 대한 편미분을 구하여라

In [7]:
# Step 1 : 상수 고정
def function_tmp1(x0): 
    return x0*x0 + 4.0**2.0

# Step 2 : x0을 기준으로 미분 진행
numerical_difference(function_tmp1,3)

6.00000000000378

문제:  
$ x_0 = 3, x_1 = 4 $ 일때, $ x_1 $에 대한 편미분을 구하여라

In [8]:
# Step 1 : 상수 고정
def function_tmp2(x1): 
    return 3.0**2.0 + x1*x1

# Step 2 : x0을 기준으로 미분 진행
numerical_difference(function_tmp2,4)

7.999999999999119

## 4.4 기울기

4.3에서는 $x_0, x_1$ 각각에 대해 변수별로 따졌었음  
하지만 실제로는 편미분을 동시에 계산하는것이 효율적  
이때, $(\frac{\partial f}{\partial x_0},\frac{\partial f}{\partial x_1})$ 처럼 모든 변수의 편미분을 벡터로 정리한 것을 **기울기**라고함  

In [14]:
def numerical_gradient(f,x): #f에는 함수 x에는 array 받음
    h = 1e-4
    grad = np.zeros_like(x)
    for idx in range(x.size):
        tmp_val = x[idx]
        #f(x+h)계산
        x[idx] = tmp_val + h
        fxh1 = f(x)

        x[idx] = tmp_val - h
        fxh2 = f(x)

        grad[idx] = (fxh1 - fxh2) / (2 * h)
        x[idx] = tmp_val
    return grad

In [15]:
import numpy as np
numerical_gradient(function_2, np.array([3.0,4.0]))

array([6., 8.])

이 기울기들은 함수의(적어도 해당 부분에서의) 가장 낮은 값을 가리킴  
즉, 기울기가 가리키는 방향은 각 장소에 함구의 출력값을 가장 크게 줄이는 방향  

##### **Gemini 활용 보충 이해**  

1. 상상해보기: 밥그릇 모양 산먼저 이 함수의 생김새를 머릿속에 그려야 합니다.$$z = x_{0}^2 + x_{1}^2$$ 그래프는 아래쪽이 둥근 '밥그릇' 모양입니다.가장 낮은 지점(목표): 바닥인 $(0, 0)$ 지점입니다. (높이 0)현재 위치: 우리가 밥그릇의 벽면 어딘가인 $(3, 4)$ 지점에 서 있다고 가정해 봅시다.현재 높이$$(손실값): 3^2 + 4^2 = 9 + 16 = \mathbf{25}$$
2. 기울기 계산 (나침반 만들기)이제 여기서 미분을 합니다. 변수가 2개니까 편미분을 해야겠죠?$x_0$ 방향 기울기: $\frac{\partial f}{\partial x_0} = 2x_0$현재 위치 $x_0=3$ 대입 $\rightarrow$ $6$의미: "동쪽($x_0$)으로 한 발짝 가면 높이가 6만큼 가파르게 올라간다."$x_1$ 방향 기울기: $\frac{\partial f}{\partial x_1} = 2x_1$현재 위치 $x_1=4$ 대입 $\rightarrow$ $8$의미: "북쪽($x_1$)으로 한 발짝 가면 높이가 8만큼 아주 가파르게 올라간다."기울기 벡터 (Gradient):이 두 숫자를 합쳐서 괄호로 묶으면 그게 바로 기울기 벡터입니다.$$(\frac{\partial f}{\partial x_0}, \frac{\partial f}{\partial x_1}) = (6, 8)$$
3. "방향이 있다"는 뜻 해석자, 이제 **$(6, 8)$**이라는 숫자가 나왔습니다. 이걸 좌표평면에 그려볼까요?가로($x_0$)로 6칸, 세로($x_1$)로 8칸 가는 화살표를 그립니다.이 화살표는 **원점 $(0,0)$의 정반대 방향(바깥쪽)**을 가리키고 있습니다.이 그림을 보면 화살표들이 전부 **"밥그릇 바깥쪽(높아지는 쪽)"**을 향해 뻗어 나가고 있죠? 이게 바로 기울기의 방향입니다.기울기 벡터 $(6, 8)$의 진짜 의미:"지금 서 있는 $(3,4)$ 지점에서 $(6, 8)$ 방향(북동쪽)으로 움직이면 산을 가장 빠르게 올라갈 수 있다."4. "가장 낮은 값을 가리킨다"는 말의 속뜻여기서 작성자님이 질문하신 문장의 비밀이 풀립니다.기울기 방향 $(6, 8)$: 함수값이 가장 크게 증가하는(산 위로 가는) 방향.기울기의 반대 방향 $(-6, -8)$: 함수값이 가장 크게 감소하는(산 아래로 가는) 방향.책에서 "기울기가 가장 낮은 곳을 가리킨다"라고 표현한 것은, 우리가 학습을 할 때 기울기에 마이너스($-$)를 붙여서 반대 방향으로 가기 때문입니다.수식으로 보면 명확합니다.$$x_{new} = x_{old} - \eta \frac{\partial f}{\partial x}$$현재 위치 $(3, 4)$기울기 $(6, 8)$ $\rightarrow$ 산 위쪽 방향빼기($-$): $(3, 4) - (6, 8) = (-3, -4)$ $\rightarrow$ 산 아래쪽(원점) 방향!즉, **"기울기(벡터) 자체는 오르막길 방향을 가리키지만, 우리는 그 정보를 이용해서 내리막길(가장 낮은 값)을 찾아낸다"** 는 뜻입니다.

### 경사하강법

학습단계에서 최적의 매개변수(가중치와 편향)을 찾아 내야함(여기서 최적이란 손실함수가 최솟값이 될때의 매개변수 값)  
**경사법**은 그 시점의 위치에서 기울어진 방향으로 일정거리만큼 이동을 하고 계산을 반복하며 함수의 값이 최소가 되는 찾아나아가는 과정임  
하지만 **기울어진 방향이 꼭 최솟값을 가리키는 것은 아님**으로 주의해야함  
경사법의 방식은 기울기가 0이 되는 지점을 찾아나아가는 것이므로  
최솟값, **안장점(saddle point)** 처럼 기울기가 0인 지점을 찾게 되면 그곳이 반드시 최솟값임을 보장할 수 없음  
실제로 대부분의 복잡한 함수는 기울기가 가리키는 방향에 최솟값이 없음

경사법을 수식으로 나타내면 다음과 같음
$$x_{0} = x_{0} - \eta \frac{\partial f}{\partial x}$$
$$x_{1} = x_{1} - \eta \frac{\partial f}{\partial x}$$
$ \eta \text{  에타}^{eta} $기호는 갱신하는 양을 나타냄  
이를 신경망 학습에서 **학습률**이라고 부름  

특징  
* 변수의 수가 늘어나도 같은 식을 사용하여 갱신
* 학습률은 미리 정해 두어야함 (0.01, 0.001등의 작은 값)
* 학습률을 변경해가며 올바르게 학습하고 있는지 확인하며 진행하며 이 값이 너무 크거나 작으면 최적의 장소를 찾지 못함

In [16]:
def gradient_descent(f, init_x, lr=0.01,step_num = 100):
    x = init_x
    for i in range(step_num):
        grad = numerical_gradient(f,x) #앞서 작성한 편미분 식
        x -= lr*grad
    return x

f는 최적화하려는 함수, init_x는 초깃값(추후 기울기가 될 것), lr은 learning rate(학습률), step_num은 경삿법 반복횟수

#### 경사법 예문

경사법으로 $ f(x_0,x_1) = x_0^2 + x_1^2 $를 구하여라

In [17]:
def function_2(x):
    return x[0]**2+x[1]**2

In [20]:
init_x = np.array([3.0,-4.4])
gradient_descent(function_2,init_x = init_x, lr = 0.01, step_num = 1000)

array([ 5.04890207e-09, -7.40505637e-09])

이 값은 0,0에 매우 가까운 값으로 실제로 진정한 최솟값이 0,0이르모 정확한 결과 구현

학습률이 너무 크거나 작은 경우 잘 작동 하지 않음 예시

In [22]:
init_x = np.array([3.0,-4.4])
print("학습률 너무 클 때 :",gradient_descent(function_2,init_x = init_x, lr = 10, step_num = 1000))
print("학습률 너무 작을 때 :", gradient_descent(function_2,init_x = init_x, lr = 1e-10, step_num = 1000))

학습률 너무 클 때 : [2.58985795e+13 1.49815380e+12]
학습률 너무 작을 때 : [2.58985795e+13 1.49815380e+12]


엄청나게 큰 값으로 발산해버림

### 신경망에서의 기울기

신경망에서는 **가중치 매개변수**에 대한 **손실함수**의 기울기를 구해야함  
$\frac{\partial L}{\partial \boldsymbol{W}}$ 의 각 원소에 관한 편미분은 다음과 같음  
$$Gradient = \frac{\partial L}{\partial W} \text{  (L: LossFunction, W: Weight)}$$

$$W = \begin{pmatrix} 
W_{11} & W_{12} & W_{13} \\ 
W_{21} & W_{22} & W_{23} 
\end{pmatrix}$$

$$\frac{\partial L}{\partial W} = \begin{pmatrix} 
\dfrac{\partial L}{\partial W_{11}} & \dfrac{\partial L}{\partial W_{12}} & \dfrac{\partial L}{\partial W_{13}} \\ 
\dfrac{\partial L}{\partial W_{21}} & \dfrac{\partial L}{\partial W_{22}} & \dfrac{\partial L}{\partial W_{23}} 
\end{pmatrix}$$

이때 1행의 첫번 째 원소는 $W_{11}$을 조금 변경했을 때 손실함수 $L$이 얼마나 변하느냐를 나타냄

$\frac{\partial L}{\partial \boldsymbol{W}}$의 형상이 $\boldsymbol{W}$와 같으므로 연산가능

In [ ]:
import sys,os
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3)

    def predict(self,x):
        return np.dot(x,self.W)

    def loss(self,x,t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y,t)
        return loss